In [21]:
MY_SERIALIZED_MODEL = './my_model/my_model_1658084748.660345.pkl'
MY_EXAMPLES = './examples.csv'
OUTPUT_PATH = './inferences.csv'

PROJECT        = ''#'gtdp-mlops-dev'
REGION         = '' #'eu'
TEMP_LOCATION  = './temp' # gs://gtdp_mlops_dev_mlinput/temp'


OUTPUT_SCHEMA  = {
                    'fields': 
                    [
                        {'name':'job_id'        ,'type':'STRING'   , 'mode': 'REQUIRED'},
                        {'name':'input_data'    ,'type':'STRING'   , 'mode': 'REQUIRED'},
                        {'name':'transform_data','type':'STRING'   , 'mode': 'NULLABLE'},
                        {'name':'output_data'   ,'type':'STRING'   , 'mode': 'NULLABLE'},
                        {'name':'tec_dat_cre'   ,'type':'TIMESTAMP', 'mode': 'REQUIRED'},
                    ]
                 }

In [22]:
%reload_ext autoreload
%autoreload 2
# testing without apache_beam
from adeo_mlops_lib import mlopslib


f = open(MY_EXAMPLES, "r")
mylist = f.read().splitlines()
mylist = [v.split(',') for v in mylist ]

input = [list(map(float, suba)) for suba in mylist]

mlopslib_x = mlopslib.load(MY_SERIALIZED_MODEL)

transform = input
predictions = mlopslib_x.predict(transform)

output = mlopslib_x.output('id',input,transform,predictions)

print(output)

del mlopslib_x


{'job_id': 'id', 'input_data': '[[4.947543583304329, 3.384978721770141, 6.253358321057868, 2.2196608276841983], [6.606564407811608, 3.387871841096402, 2.2603987851981175, 0.7687599498001358], [7.3264910684980205, 3.041734996249156, 4.064899464009686, 0.7265987299245709], [7.165454322623381, 2.0404524073848194, 6.3113436168139305, 0.4179782219957122], [7.631371622184243, 4.159669161223599, 2.108975661920023, 1.7692167487195543], [5.163505356810251, 2.2884783524037653, 5.925285117054725, 0.9345365600233835], [7.432203226221343, 3.064280638405448, 6.248381045549924, 1.058084818450686], [7.72653456529427, 2.5576219532138422, 1.36377663018207, 2.233173987273001], [6.829406912383567, 2.0239158705319276, 4.437455754817621, 1.4016064555294137], [6.122721578790571, 2.384874079943531, 4.588292611181637, 2.252758710428669]]', 'transform_data': '[[4.947543583304329, 3.384978721770141, 6.253358321057868, 2.2196608276841983], [6.606564407811608, 3.387871841096402, 2.2603987851981175, 0.7687599498001

In [23]:
import warnings
warnings.filterwarnings("ignore")

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

from apache_beam.dataframe.io import read_csv
from apache_beam.dataframe.convert import to_dataframe
from apache_beam.dataframe.convert import to_pcollection

import apache_beam.runners.interactive.interactive_beam as ib

from adeo_mlops_lib import mlopslib

mlopslib_x = mlopslib.load(MY_SERIALIZED_MODEL)

class Predict(beam.DoFn):
  def process(self, input):
    transform = mlopslib_x.transform(input) 
    inference = mlopslib_x.predict(transform)
    yield mlopslib_x.output('id', input, transform, inference) # converts the output to the desired format


beam_options = {}
#p = beam.Pipeline(InteractiveRunner())

p = beam.Pipeline(options=PipelineOptions(beam_options, project=PROJECT, region=REGION, temp_location=TEMP_LOCATION))

batch_rows = (
    p 
    | 'ReadLocal'  >> beam.io.ReadFromText(MY_EXAMPLES)
    | 'Split'      >> beam.Map(lambda record: record.split(','))
    | 'Predict'    >> beam.ParDo(Predict())
    | 'WriteLocal' >> beam.io.WriteToText(OUTPUT_PATH)
)

p.run()